In [1]:
sodauri = 'https://soda.demo.socrata.com/resource/6yvf-kk3n.json'

In [2]:
sodainputDf = spark.read.option('header', 'true').csv('./sodainput.csv')

In [3]:
sodainputDf.createOrReplaceTempView('sodainputtbl')

In [4]:
prmsSoda = { 'url' : sodauri, 'input' : 'sodainputtbl', 'method' : 'GET', 'readTimeout' : '10000', 'connectionTimeout' : '2000', 'partitions' : '10'}

In [5]:
sodasDf = spark.read.format("org.apache.dsext.spark.datasource.rest.RestDataSource").options(**prmsSoda).load()

In [6]:
sodasDf.printSchema() 

root
 |-- output: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- depth: string (nullable = true)
 |    |    |-- earthquake_id: string (nullable = true)
 |    |    |-- location: struct (nullable = true)
 |    |    |    |-- coordinates: array (nullable = true)
 |    |    |    |    |-- element: double (containsNull = true)
 |    |    |    |-- type: string (nullable = true)
 |    |    |-- magnitude: string (nullable = true)
 |    |    |-- number_of_stations: string (nullable = true)
 |    |    |-- region: string (nullable = true)
 |    |    |-- source: string (nullable = true)
 |    |    |-- version: string (nullable = true)
 |-- region: string (nullable = true)
 |-- source: string (nullable = true)



In [7]:
sodasDf.createOrReplaceTempView("sodastbl")

In [17]:
spark.sql("describe sodastbl").show()

+--------+--------------------+-------+
|col_name|           data_type|comment|
+--------+--------------------+-------+
|  output|array<struct<dept...|   null|
|  region|              string|   null|
|  source|              string|   null|
+--------+--------------------+-------+



In [10]:
spark.sql("select source,region, inline(output) from sodastbl").show()

+------+--------------------+-----+-------------+--------------------+---------+------------------+--------------------+------+-------+
|source|              region|depth|earthquake_id|            location|magnitude|number_of_stations|              region|source|version|
+------+--------------------+-----+-------------+--------------------+---------+------------------+--------------------+------+-------+
|    nn|              Nevada|  9.2|     00388520|[[-117.938, 38.66...|      2.3|                 9|              Nevada|    nn|      9|
|    nn|              Nevada|  4.1|     00388527|[[-115.701, 36.23...|      1.1|                15|              Nevada|    nn|      9|
|    nn|              Nevada|  8.5|     00388458|[[-118.4548, 38.4...|      1.6|                13|              Nevada|    nn|      9|
|    nn|              Nevada|    0|     00388285|[[-117.4526, 41.2...|      3.9|                 6|              Nevada|    nn|      9|
|    nn|              Nevada|  7.5|     00388209

In [11]:
spark.sql("select output.depth from sodastbl").show()

+--------------------+
|               depth|
+--------------------+
|[9.2, 4.1, 8.5, 0...|
|[25, 43, 47, 43, ...|
|                  []|
+--------------------+



In [15]:
spark.sql("select source, region, inline(output.depth) from sodastbl").show()

AnalysisException: cannot resolve 'inline(sodastbl.`output`.`depth`)' due to data type mismatch: input to function inline should be array of struct type, not array<string>; line 1 pos 23;
'Project [source#49, region#48, unresolvedalias(inline(output#47.depth), None)]
+- SubqueryAlias sodastbl
   +- Relation[output#47,region#48,source#49] RESTRelation(https://soda.demo.socrata.com/resource/6yvf-kk3n.json)
